# Connect X Project

In this game, your objective is to get a certain number of your checkers in a row horizontally, vertically, or diagonally on the game board before your opponent. When it's your turn, you “drop” one of your checkers into one of the columns at the top of the board. Then, let your opponent take their turn. This means each move may be trying to either win for you, or trying to stop your opponent from winning. The default number is four-in-a-row, but we’ll have other options to come soon.

# Import Allowed Libraries

In [362]:
import scipy
import numpy as np
import random
import time

# Board Class

In [303]:
class Board:
    """Class representing the ConnectX Board"""
    
    def __init__(self, level, branch, parent=None, state=None, n_rows=6, n_cols=7, game_type=4, win_state=None):
        
        self.n_rows = n_rows
        self.n_cols = n_cols
        self.game_type = game_type
        self.win_state = win_state
        self.level = level
        self.branch = branch
        self.parent = parent
        
        if state == None:
            self.state = np.zeros(shape=[self.n_rows, self.n_cols])
        else:
            self.state = state
    
            
    def update_board(self, column, player):
        """
        Drop the specified player's chip into the chosen column
        
        column: 0 <= column < n_cols
        player: [1, 2]
        
        
        """
        if self.win_state == 0:
            print('Draw')
            return None
        elif self.win_state == 1:
            print('Player 1 Won')
            return None
        elif self.win_state == 2:
            print('Player 2 Won')
            return None
        
        if self.check_full_col(column=column):
            print(f'Column {column} is full')
            return None
        
        for row, row_val in enumerate(self.state[:, column], 0):
            if row_val == 0:
                print(f'Player {player}\'s chip dropped onto column {column}, row {row}')
                self.state[row, column] = player
                break
        
        # check for win
        self.check_win()
        
    def check_vert_line(self):
        """
        Check if there are 4 connected chips in a vertical line.
        """
        
        r_lim = self.n_rows - self.game_type  # limit for which row we check till. past the r_lim row, not possible to extend grid vertically up
        for c in range(self.n_cols):
            for r in range(r_lim + 1):
                cell = self.state[r, c]
                
                if cell == 1: # if p1 chip is in [r, c]
                    vert_group = self.state[r:r+4, c] # isolate the 4-cell rectangle extending up from the p1 chip cell
                    if np.all(vert_group == 1): # if all 4 cells have the same value (p1 chip), return win.
                        print('Player 1 Wins')
                        self.win_state = 1
                        return None
                        
                elif cell == 2: # if p2 chip is in [r, c]
                    vert_group = self.state[r: r+4, c] # isolate the 4-cell rectangle extending up from the p1 chip cell
                    if np.all(vert_group == 2): # if all 4 cells have the same value (p1 chip), return win.
                        print('Player 2 Wins')
                        self.win_state = 2
                        return None

        print('No Vert Lines Found')
        
        
        
    def check_horizontal_line(self):
        """
        Check if there are 4 connected chips in a horizontal line.
        """
        c_lim = self.n_cols - self.game_type # limit for which column we check till. past the c_lim column, not possible to extend grid horizontally to the right
        for c in range(0, c_lim + 1):
            for r in range(self.n_rows):
                cell = self.state[r, c]
                
                if cell == 1: # if p1 chip is in [r, c]
                    hori_group = self.state[r, c:c+4] # isolate the 4-cell rectangle extending up from the p1 chip cell
                    if np.all(hori_group == 1): # if all 4 cells have the same value (p1 chip), return win.
                        print('Player 1 Wins')
                        self.win_state = 1
                        return None
                        
                elif cell == 2: # if p2 chip is in [r, c]
                    hori_group = self.state[r, c:c+4] # isolate the 4-cell rectangle extending up from the p1 chip cell
                    if np.all(hori_group == 2): # if all 4 cells have the same value (p1 chip), return win.
                        print('Player 2 Wins')
                        self.win_state = 2
                        return None

        print('No Horizontal Lines Found')
        

    def check_diagonal_line(self):
        """
        Check if there are 4 connected chips in a diagonal line.
        """

        for c in range(self.n_cols):
            for r in range(self.n_rows):
                cell = self.state[r, c]
                
                if cell == 1:
                    # check / diagonals
                    if not (r + 4 >= self.n_rows) and not (c + 4 > self.n_cols):
                        up_right_group = []
                        for i in range(self.game_type):
                            up_right_group.append(self.state[r+i, c+i])
                        up_right_group = np.array(up_right_group)
                        
                        if np.all(up_right_group == 1):
                            print('Player 1 Wins')
                            self.win_state = 1
                            return None
                        
                    
                    if not (r - 3 < 0) and not (c + 4 > self.n_cols):
                        down_right_group = []
                        for i in range(self.game_type):
                            down_right_group.append(self.state[r-i, c+i])
                            
                        down_right_group = np.array(down_right_group)
 
                        if np.all(down_right_group == 1):
                            print('Player 1 Wins')
                            self.win_state = 1
                            return None
                        
                elif cell == 2:
                    # check / diagonals
                    if not (r + 3 >= self.n_rows) and not (c + 4 > self.n_cols):
                        up_right_group = []
                        for i in range(self.game_type):
                            up_right_group.append(self.state[r+i, c+i])
                        up_right_group = np.array(up_right_group)
                        
                        if np.all(up_right_group == 2):
                            print('Player 2 Wins')
                            self.win_state = 2
                            return None
                        
                    # check \ diagonals
                    if not (r - 3 < 0) and not (c + 4 > self.n_cols):
                        down_right_group = []
                        for i in range(self.game_type):
                            down_right_group.append(self.state[r-i, c+i])
                        down_right_group = np.array(down_right_group)
                        
                        if np.all(down_right_group == 2):
                            print('Player 2 Wins')
                            self.win_state = 2
                            return None
                        
        print('No Diagonal Lines Found')
        
    def check_win(self):
        """
        Check if a player has won.
        """
        self.check_vert_line()
        self.check_horizontal_line()
        self.check_diagonal_line()
    
        if self.win_state == 1:
            print('Player 1 Wins')
        elif self.win_state == 2:
            print('Player 2 Wins')
        elif self.win_state == 0:
            print('Draw! The board is full.')

        return None
    
    def check_draw(self):
        """
        Check if the board is full.
        """
        
        pass
    
    def check_full_col(self, column):
        """
        Check if a column is full
        """
        
        if np.all(self.state[:, column] != 0):
            return True
        else:
            return False

# Running Connect X Games

Now that our board class is ready, we can simulate random games by using the methods of the class. Here, we initialise a Connect 4 board and run random moves for 2 players till we have a winner.

In [370]:
board = Board(level=0, branch=0)
i = 1

In [372]:
while board.win_state is None:

    if i % 2 != 0:
        col = random.randint(0, 6)
        while board.check_full_col(col):
            col = random.randint(0, 6)
        board.update_board(column=col, player=1)
        i += 1

    elif i % 2 == 0:
        while board.check_full_col(col):
            col = random.randint(0, 6)
        board.update_board(column=col, player=2)
        i += 1

    print(board.state, end='\r')
    time.sleep(1)

Player 1's chip dropped onto column 0, row 2
No Vert Lines Found
No Horizontal Lines Found
No Diagonal Lines Found
[[1. 0. 1. 0. 0. 1. 0.]
 [2. 0. 2. 0. 0. 2. 0.]
 [1. 0. 1. 0. 0. 0. 0.]
 [0. 0. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
Player 2's chip dropped onto column 0, row 3
No Vert Lines Found
No Horizontal Lines Found
No Diagonal Lines Found
[[1. 0. 1. 0. 0. 1. 0.]
 [2. 0. 2. 0. 0. 2. 0.]
 [1. 0. 1. 0. 0. 0. 0.]
 [2. 0. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
Player 1's chip dropped onto column 4, row 0
No Vert Lines Found
No Horizontal Lines Found
No Diagonal Lines Found
[[1. 0. 1. 0. 1. 1. 0.]
 [2. 0. 2. 0. 0. 2. 0.]
 [1. 0. 1. 0. 0. 0. 0.]
 [2. 0. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
Player 2's chip dropped onto column 4, row 1
No Vert Lines Found
No Horizontal Lines Found
No Diagonal Lines Found
[[1. 0. 1. 0. 1. 1. 0.]
 [2. 0. 2. 0. 2. 2. 0.]
 [1. 0. 1. 0. 0. 0. 0.]
 [2. 0. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
Player 1's chip dropped onto column 1, row 0
No Vert Lines F